In [1]:
#Define the path of the model weights to be loaded
model_path = "/kaggle/input/ai_gardenwise_weights/keras/default/1/gemma_garden_wise_weights_ds939.weights.h5" # model fine-tuned with 939 instances

In [2]:
# Install packages

!pip install --upgrade tensorflow
!pip install --upgrade keras
!pip install --upgrade keras-nlp
!pip install --upgrade tensorflow-text

# Import packages

import os
import pandas as pd
import numpy as np
import json
import tensorflow_text as text
import tensorflow as tf

# Import Keras and Keras NLP from TensorFlow

from tensorflow import keras
import keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 735.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but y

2024-11-14 14:53:50.722333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731596030.745914      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731596030.753139      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Use TensorFlow as Backend

os.environ["KERAS_BACKEND"] = "tensorflow"

In [4]:
from keras_nlp.models import GemmaCausalLM

In [5]:
# Step 1: Recreate the model architecture

model = GemmaCausalLM.from_preset("gemma2_2b_en")

2024-11-14 14:54:15.482904: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [6]:
model.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Step 2: Load the model

#model_path = '/content/drive/MyDrive/gemma_garden_wise_weights.weights.h5'

# Check if the file exists

if not os.path.exists(model_path):
    print(f"Error: Weights file not found at {model_path}")
else:
    try:
        # Attempt loading with tf.keras.utils.load_weights

        import tensorflow as tf
        with tf.keras.utils.custom_object_scope({'ReversibleEmbedding': keras_nlp.layers.ReversibleEmbedding}):
            model.load_weights(model_path)
        print("Weights loaded successfully using `load_weights`!")
    except ValueError:
        print("Error loading weights with `load_weights`. Trying `tf.keras.models.load_model`")

        # If load_weights fails, attempt loading with tf.keras.models.load_model

        with tf.keras.utils.custom_object_scope({'ReversibleEmbedding': keras_nlp.layers.ReversibleEmbedding}):
            model = tf.keras.models.load_model(model_path)
        print("Weights loaded successfully using `load_model`!")

Weights loaded successfully using `load_weights`!


/usr/local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [8]:
# Step 3: Configure the AdamW optimizer with weight decay exclusions

optimizer = keras.optimizers.AdamW(

    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# Step 4: Compile the model

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Verify the model

model.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

## MODEL EVALUATION

In [8]:
# Path for the test dataset

test_data_path = "/kaggle/input/ai-gardenwise-test-dataset/GardenWise_test_dataset_6.jsonl"

In [9]:
# Define a function to read the jsonl file

def read_jsonl(file_path):

    with open(file_path, 'r', encoding='utf-8') as file:

        for line in file:

            yield json.loads(line)

In [10]:
!pip install -U nltk
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.4 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=40e15bb836237e668f7fb29fb6cf4f6d95440cd889fd2dda0900ca8425faceb9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import json
import nltk

nltk.download('punkt')
nltk.download('wordnet', quiet=True)

# Define scoring objects and smoothing function
smoothing_function = SmoothingFunction().method1
rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Initialize cumulative scores and counters
total_bleu, total_rouge1, total_rougeL, total_meteor_score, count = 0, 0, 0, 0, 0

# Load test data (assuming 'read_jsonl' is a defined function for loading)
test_data = read_jsonl(test_data_path)

# Iterate over the test data, generate responses, and calculate all scores
for entry in test_data:
    prompt = f"Instruction: {entry['instruction']}\n\nResponse:"
    expected_response = entry['response']  # Assumed to exist in your test data structure

    # Generate a response using the model
    generated_response = model.generate(prompt, max_length=256)

    # Calculate BLEU score with smoothing
    bleu_score = sentence_bleu([expected_response.split()], generated_response.split(), smoothing_function=smoothing_function)
    total_bleu += bleu_score

    # Calculate ROUGE scores
    rouge_scores = rouge.score(expected_response, generated_response)
    total_rouge1 += rouge_scores['rouge1'].fmeasure
    total_rougeL += rouge_scores['rougeL'].fmeasure

    # Calculate METEOR score
    meteor = meteor_score([expected_response.split()], generated_response.split())
    total_meteor_score += meteor

    # Increment counter
    count += 1

# Calculate average scores
average_bleu = total_bleu / count if count > 0 else 0
average_rouge1 = total_rouge1 / count if count > 0 else 0
average_rougeL = total_rougeL / count if count > 0 else 0
average_meteor_score = total_meteor_score / count if count > 0 else 0

# Print evaluation results
print(f"Average BLEU Score: {average_bleu}")
print(f"Average ROUGE-1 F1 Score: {average_rouge1}")
print(f"Average ROUGE-L F1 Score: {average_rougeL}")
print(f"Average METEOR Score for Role Test Dataset: {average_meteor_score}")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
I0000 00:00:1731596322.403442     294 service.cc:148] XLA service 0x7fec503c94f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731596322.403514     294 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1731596322.410898     294 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Average BLEU Score: 0.023133803553489995
Average ROUGE-1 F1 Score: 0.2861277301883921
Average ROUGE-L F1 Score: 0.20434017986247388
Average METEOR Score for Role Test Dataset: 0.25608774015534047


In [13]:
# Inferences

import random


# Load test data

test_data = list(read_jsonl(test_data_path))  # Convert to a list to enable random sampling


# Randomly select 5 entries

#entries = random.sample(test_data, 5)
# or, for the first 3 entries:
entries = test_data[0:3]

# Iterate over the randomly selected entries and generate responses

for entry in entries:
    prompt = f"Instruction: {entry['instruction']}\n\nResponse:"
    expected_response = entry['response']

    # Generate a response using the model
    generated_response = model.generate(prompt, max_length=256)

    # Print the expected and generated responses
    print(f"Instruction: {entry['instruction']}")
    print(f"Expected Response: {expected_response}")
    print()
    print(f"Generated Response: {generated_response}")
    print("\n" + "-"*50 + "\n")


Instruction: What are the easiest vegetables to grow at home?
Expected Response: Lettuce, radishes, green beans, and cherry tomatoes are easy vegetables to grow at home. They need minimal care, sunlight, and regular watering, making them great for beginners and small spaces.

Generated Response: Instruction: What are the easiest vegetables to grow at home?

Response: The easiest vegetables to grow at home are lettuce, radishes, and carrots. These crops are fast-growing and require minimal maintenance, making them ideal for beginners. Lettuce, in particular, is a popular choice for home gardeners due to its versatility and ease of cultivation.

--------------------------------------------------

Instruction: How often should I water mybgarden?
Expected Response: Water your garden once or twice a week, allowing the soil to dry slightly between waterings.

Generated Response: Instruction: How often should I water mybgarden?

Response: Water your garden regularly, especially during hot wea